## Important Update (2019-07-02):
The content in this notebook is no longer up to date. It is kept here only for reference.

Please refer to the `Web3 Examples` and `hello_luce` notebooks instead for examples on how to use Python (and LuceVM) to interact programatically with the Ethereum blockchain.

# Example of Interaction between Python Code & Ethereum Smart Contract

This notebook outlines how to deploy and interact with an Ethereum Solidity Smart Contract using the Python Web3 framework. It covers the setup, as well as deployment and interaction with a contract locally via Ganache. These instructions come in both `.pdf` and `.ipynb` format. The PDF file is intended to access the instructions while setting up the working environment. Once the virtual environment and jupyter notebook server are running the instructions can then be followed interactively directly from the Jupyter notebook file. 

Software used:  
* [Python 3.6](https://www.python.org)
* [Jupyter](https://jupyter.org)
* [Ganache](https://www.trufflesuite.com/ganache)
* [Remix](https://remix.ethereum.org)

### Preparation
I assume that Python 3.6 (3.7 or higher should work equally well), Ganache and the Jupyter notebook (for example via Anaconda) are already installed on the system. If not, please install these components via the instructions provided on the respective websites.  

Next, we create a new project folder somewhere on the system and place the `luce_python.ipynb` file inside this folder.

### Setup Python Virtualenv

It is good practice to set up a separate isolated python development environment for each project. This avoids library version conflicts and also helps with reproducibility as the same working environment can be easily recreated in the future. To set up a new virtual python environment we use the following commands in the terminal:

Navigate to desired project location:  
(Note: This directory must already exist.)   
```cd /Users/arno/Desktop/luce_python/```

Install virtual environment manager:  
```pip install virtualenv```. 

Create new virtual environment:  
```virtualenv .luce_python```

Activate the new environment:  
```source .luce_python/bin/activate```

Install jupyter kernel in new environment:  
```pip install ipykernel```  
```python -m ipykernel install --user --name=luce_python_kernel```

Run jupyter notebook  
```jupyter notebook```


### Connect Jupyter Notebook to Python Kernel

Now, inside the Jupyter Noebook environment we open the `luce_python.ipynb` file to continue with the instructions. We change the python kernel to match the new environment we just created via `Kernel -> Change Kernel -> luce_python _kernel`. We then execute the python cell below to verify that the notebook is indeed using the correct python environment. The path displayed should match the one of our project folder. For example: `/Users/arno/Desktop/luce_python/.luce_python/bin/python`

In [1]:
# Check that correct python kernel is running in Notebook
import sys
sys.executable
#sys.prefix

'/home/vagrant/miniconda/envs/luce_vm/bin/python'

In [2]:
!which python

/home/vagrant/miniconda/envs/luce_vm/bin/python


Python version used by shell is the same as python kernel used by Jupyter notebook. Very good! :)

### Install Python Libraries
We install the Python web3 framework and python solidity compiler. The special syntax is used to execute terminal commands from within jupyter and ensure that the packages are installed into our virtual python environment.

In [4]:
# Install Python Web3 package
#!{sys.executable} -m pip install web3

  Using cached https://files.pythonhosted.org/packages/24/f5/3d6e3ac1ec98dc736084a8f821b922517d5492714889bf29460d9d97bda0/web3-4.9.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a3/d8/6118bc0e00d6ec092bb02b012989d1018893755eb84da53266412ac359e3/eth_account-0.3.0-py3-none-any.whl
     |████████████████████████████████| 61kB 6.2MB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/4e/99/7e0c9a2f74f2a0b4512bd4491334537f4a3418a4daec6ae90ed5cd8d6b89/hexbytes-0.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0c/6a/d048dfe820fc956e57bb1115f5eda5a1bef320172811f72c9924c8d6ebb5/websockets-6.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/0d/d8/0f0c8d4ccaa4c8d25524fd1ed9f5c6d9551b52b74b973f338de3f0a23111/eth_hash-0.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e9/ef/5af786a32ae7ac3aece6489400ea2cee6b2ede7a7b3955e6f55a59442d12/eth_abi-1.3.0-py3-none-any.

In [ ]:
# Install Python Solidity Compiler
#!{sys.executable} -m pip install py-solc

### Solidity Smart Contract

```javascript
pragma solidity ^0.4.26;

contract Data{
    
    address public dataProvider;
    uint public licence;
    string private link;
    string public dataDescription="default"; //this needs to become a struct!

    // The keyword "public" makes those variables
    // easily readable from outside.
    mapping (address => uint) userTokens;
    mapping (address => address) public mappedUsers;
    address[] public addressIndices;
    // Events allow light clients to react to changes efficiently.
    event Sent(address from, address to, uint token);
    event publishedDataset(address publisher, string description, string link, uint licence); // Event
    event updateDataset(address to, string uspdateDescr, string link);
    constructor () public{
                dataProvider=msg.sender;
    }
    
    function publishData(string memory _newdescription, string memory _link, uint _licence) public {
        require(msg.sender == dataProvider);
        dataDescription=_newdescription;
        link=_link;
        licence=_licence;
        emit publishedDataset(msg.sender, _newdescription, link, licence); // Triggering event
    }
    function setLicence(uint newLicence) public{
       dataProvider=msg.sender;
       licence=newLicence;
    //TODO we need to update all of the changes!! 
    //Not focusing here as I am not sure we need to change licences once the data is published.
    }
    
    
    function getLicence() public view returns(uint) {
       return licence;
    }
    //DataRequesters get the link to the data only if the token is right!
    function getLink(uint token) public view returns(string memory){
        require(token==1);
        return link;
    }
    //This is a function to notify the dataRequesters to update the data records
    function updateData(string memory updateDescr, string memory _newlink) public{
        require(dataProvider==msg.sender);
        dataDescription=updateDescr;
        link=_newlink;
        uint arrayLength = addressIndices.length;
        for (uint i=0; i<arrayLength; i++) {
            address to=mappedUsers[addressIndices[i]];
            emit updateDataset(to, updateDescr, link); // Triggering event for all dataRequesters
        }//for
   }
    function addDataRequester(uint purposeCode, uint licenceType) public returns(uint){
       //for now the purpose is a code as the string comparison it's expensive in solidity
       //in the future the purpose should be compared to a field of the overall contract description
        require(purposeCode<=20);
        require(licence==licenceType);
        addressIndices.push(msg.sender); //adding the data requester to an array so that I can loop the mapping of dataRequesters later!
        mappedUsers[msg.sender] = msg.sender;//adding a new data requester (key and value are the same!)!
        userTokens[msg.sender] = 1; //TODO this should become a token generation function!
        uint token=1; //TODO this is a shortcut. tokens should be derived from some verifiable function that cannot be faked
        return token;
    }
    function renewToken(uint compliance) public returns(uint token){
        require(userTokens[msg.sender] > 0, "need to agree on licence first");
        if(licence==compliance){
            emit Sent(msg.sender, dataProvider, userTokens[msg.sender]++);
           token=userTokens[msg.sender]++;
           //TODO add compliance in respect to dataUpdates
           //the compliance that is given in input will need to show that updates were performed
        }
        else{
            userTokens[msg.sender] = 0;
            emit Sent(msg.sender, dataProvider, 0);
            token=0;
        }
        return token;
    }
}
```

### Deploy Contract

Compile (Compiler Version: 0.4.26 commit) & Deploy the solidity contract via Remix and Chrome + Metamask web3 injection to locally running instance of Ganache. Obtain smart contract address from transaction within Ganache and the public and private wallet keys from Metamask.

### Connect Python to local Ganache Node

In [5]:
import time
from web3 import Web3, HTTPProvider

# Contract address from Ganache
contract_address     = "0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E"

# Private key obtained via Metamask
wallet_private_key   = "46CFCDA83AAA94D428D971517AC4A80AEE039B5C4B5CC6D876B78D06076A025A"

# Wallet address from Metamask
wallet_address       = "0x0a68a27F5b59C7Ed8A3c64330745AffD88ecfc85"

# Ganache Connection
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))

### Interacting with the smart contract

In order to interact with the contract functions we need to expose the application binary interface (ABI) of the compiled smart contract to our Python code. The easiest way to do so is by copying the ABI data in json from Remix and storing it in another python module for easy access. (The ABI can be found in the Compile tab in Remix after compilation of the smart contract.)

In [3]:
# Import Python module containing application binary interface 
import luce_abi
# Instantiate web3 contract object using the contract address & corresponding application binary interface
contract = w3.eth.contract(address = contract_address, abi = luce_abi.abi)

This does NOT work:

In [4]:
def setLicence(licence_uint):
    return contract.functions.setLicence(licence_uint).call()

But this DOES work:

In [5]:
def setLicence(licence_uint):
    nonce = w3.eth.getTransactionCount(wallet_address)
    txn_dict = contract.functions.setLicence(licence_uint).buildTransaction({
        'chainId': 3,
        'gas': 140000,
        'gasPrice': w3.toWei('40', 'gwei'),
        'nonce': nonce,
    })
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(txn_dict, private_key=wallet_private_key)    

    # Send transaction and store hash
    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Check if transaction was added to blockchain
    time.sleep(2) # wait 2 seconds
    tx_receipt = w3.eth.getTransactionReceipt(txn_hash)
    if tx_receipt is None:
        return {'status': 'failed', 'error': 'timeout'}
    else:
        print(tx_receipt)

In [6]:
setLicence(3)

AttributeDict({'transactionHash': HexBytes('0x80d20f65007d3a7370877d1d4d76f95e48b42fc19a6e1b51795ac4d6077758b2'), 'transactionIndex': 0, 'blockHash': HexBytes('0x7ffdfa09465347e1da19e2d0aa7bd5f1d59dbcca54963b7fb41db637d55366ed'), 'blockNumber': 31, 'from': '0x0a68a27f5b59c7ed8a3c64330745affd88ecfc85', 'to': '0x9b3da536bfff54974ae3d9151d7c6f5dbe81990e', 'gasUsed': 47001, 'cumulativeGasUsed': 47001, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'v': '0x

In [7]:
def getLicence():
    return contract.functions.getLicence().call()

In [8]:
getLicence()

3

---

In [9]:
def publishData(description, link, licence):
    nonce = w3.eth.getTransactionCount(wallet_address)
    txn_dict = contract.functions.publishData(description, link, licence).buildTransaction({
        'chainId': 3,
        'gas': 140000,
        'gasPrice': w3.toWei('40', 'gwei'),
        'nonce': nonce,
    })
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(txn_dict, private_key=wallet_private_key)    

    # Send transaction and store hash
    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    # Check if transaction was added to blockchain
    time.sleep(2) # wait 2 seconds
    tx_receipt = w3.eth.getTransactionReceipt(txn_hash)
    if tx_receipt is None:
        return {'status': 'failed', 'error': 'timeout'}
    else:
        print(tx_receipt)

In [10]:
publishData("test description","test link",3)

AttributeDict({'transactionHash': HexBytes('0x27353b68c53891a44194cc46722b318fc4e951208ca5ccf20e93b415cbba2e0c'), 'transactionIndex': 0, 'blockHash': HexBytes('0x667fd72c3097bc5b33660a7da69c51500345b9b9c1c2be563d29f67835d2cf04'), 'blockNumber': 32, 'from': '0x0a68a27f5b59c7ed8a3c64330745affd88ecfc85', 'to': '0x9b3da536bfff54974ae3d9151d7c6f5dbe81990e', 'gasUsed': 65548, 'cumulativeGasUsed': 65548, 'contractAddress': None, 'logs': [AttributeDict({'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x27353b68c53891a44194cc46722b318fc4e951208ca5ccf20e93b415cbba2e0c'), 'blockHash': HexBytes('0x667fd72c3097bc5b33660a7da69c51500345b9b9c1c2be563d29f67835d2cf04'), 'blockNumber': 32, 'address': '0x9B3da536bfFf54974AE3D9151D7C6F5dBE81990E', 'data': '0x0000000000000000000000000a68a27f5b59c7ed8a3c64330745affd88ecfc85000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000c000000000000000000000000000000000000000000